In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import os


from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support

In [2]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [3]:
df1 = pd.read_csv("D:\data science\project\cradict risk project\case_study1.xlsx - case_study1.csv")
df2 = pd.read_csv("D:\data science\project\cradict risk project\case_study2.xlsx - case_study2.csv")
df3 = pd.read_csv("D:\data science\project\cradict risk project\Features_Target_Description.xlsx - Sheet1.csv")

In [7]:
df1.head(5)

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
0,1,5,4,1,0,0,0.000,0.0,0.200,0.800,...,0,0,1,0,4,1,4,0,72,18
1,2,1,0,1,0,0,0.000,0.0,1.000,0.000,...,0,1,0,0,0,0,1,0,7,7
2,3,8,0,8,1,0,0.125,0.0,1.000,0.000,...,0,6,1,0,0,2,6,0,47,2
3,4,1,0,1,1,0,1.000,0.0,1.000,0.000,...,0,0,0,0,0,0,1,1,5,5
4,5,3,2,1,0,0,0.000,0.0,0.333,0.667,...,0,0,0,0,0,3,0,2,131,32


In [8]:
df2.head(5)

,PROSPECTID,time_since_recent_payment,time_since_first_deliquency,time_since_recent_deliquency,num_times_delinquent,max_delinquency_level,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,...,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,max_unsec_exposure_inPct,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,549,35,15,11,29,29,0,0,0,...,0.0,0.0,0.0,13.333,1,0,PL,PL,696,P2
1,2,47,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,0.860,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,302,11,3,9,25,25,1,9,8,...,0.0,0.0,0.0,5741.667,1,0,ConsumerLoan,others,693,P2
3,4,-99999,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,9.900,0,0,others,others,673,P2
4,5,583,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,-99999.000,0,0,AL,AL,753,P1


In [10]:
df3.head()

,Case_study1 table,Variable Name,Description
0,Internal,Total_TL,Total trade lines/accounts in Bureau
1,NaN,Tot_Closed_TL,Total closed trade lines/accounts
2,NaN,Tot_Active_TL,Total active accounts
3,NaN,Total_TL_opened_L6M,Total accounts opened in last 6 Months
4,NaN,Tot_TL_closed_L6M,Total accounds closes in last 6 months


In [12]:
print(df1.shape)
print(df2.shape)
print(df3.shape)

(51336, 26)
(51336, 62)
(88, 3)


In [14]:
df1 = df1.loc[df1["Age_Oldest_TL"]!=-99999]

In [15]:
# remove null values columns

columns_to_be_remove=[]

for i in df2.columns:
    if df2.loc[df2[i] == -99999].shape[0] > 10000:
        columns_to_be_remove.append(i)

In [16]:
df2 = df2.drop(columns_to_be_remove, axis=1)

In [17]:
for i in df2.columns:
    df2 = df2.loc[df2[i]!= -9999]

In [18]:
print(df1.isnull().sum())
print(df2.isnull().sum())

PROSPECTID              0
Total_TL                0
Tot_Closed_TL           0
Tot_Active_TL           0
Total_TL_opened_L6M     0
Tot_TL_closed_L6M       0
pct_tl_open_L6M         0
pct_tl_closed_L6M       0
pct_active_tl           0
pct_closed_tl           0
Total_TL_opened_L12M    0
Tot_TL_closed_L12M      0
pct_tl_open_L12M        0
pct_tl_closed_L12M      0
Tot_Missed_Pmnt         0
Auto_TL                 0
CC_TL                   0
Consumer_TL             0
Gold_TL                 0
Home_TL                 0
PL_TL                   0
Secured_TL              0
Unsecured_TL            0
Other_TL                0
Age_Oldest_TL           0
Age_Newest_TL           0
dtype: int64
PROSPECTID                    0
time_since_recent_payment     0
num_times_delinquent          0
max_recent_level_of_deliq     0
num_deliq_6mts                0
num_deliq_12mts               0
num_deliq_6_12mts             0
num_times_30p_dpd             0
num_times_60p_dpd             0
num_std                

In [19]:
# checking comon column names

for i in list(df1.columns):
    if i in list(df2.columns):
        print(i)

PROSPECTID


In [20]:
# PROSPECTID = coustomerID

In [21]:
# Marge the two dataframes, inner join  so that no null are present

df = pd.merge(df1, df2, how = "inner", on=["PROSPECTID"])

In [22]:
df

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,5,4,1,0,0,0.000,0.00,0.200,0.800,...,0.0,0.0,0.0,0.0,1,0,PL,PL,696,P2
1,2,1,0,1,0,0,0.000,0.00,1.000,0.000,...,0.0,0.0,0.0,0.0,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,8,0,8,1,0,0.125,0.00,1.000,0.000,...,0.0,0.0,0.0,0.0,1,0,ConsumerLoan,others,693,P2
3,4,1,0,1,1,0,1.000,0.00,1.000,0.000,...,0.0,0.0,0.0,0.0,0,0,others,others,673,P2
4,5,3,2,1,0,0,0.000,0.00,0.333,0.667,...,0.0,0.0,0.0,0.0,0,0,AL,AL,753,P1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51291,51332,3,0,3,1,0,0.333,0.00,1.000,0.000,...,0.0,0.0,0.0,0.0,0,0,ConsumerLoan,ConsumerLoan,650,P4
51292,51333,4,2,2,0,1,0.000,0.25,0.500,0.500,...,0.0,0.0,0.0,0.0,0,0,others,others,702,P1
51293,51334,2,1,1,1,1,0.500,0.50,0.500,0.500,...,1.0,0.0,1.0,0.0,0,0,ConsumerLoan,others,661,P3
51294,51335,2,1,1,0,0,0.000,0.00,0.500,0.500,...,0.0,0.0,0.0,0.0,0,0,ConsumerLoan,others,686,P2


In [24]:
df.shape

(51296, 79)

In [25]:
df.isnull().sum()

PROSPECTID             0
Total_TL               0
Tot_Closed_TL          0
Tot_Active_TL          0
Total_TL_opened_L6M    0
                      ..
GL_Flag                0
last_prod_enq2         0
first_prod_enq2        0
Credit_Score           0
Approved_Flag          0
Length: 79, dtype: int64

In [26]:
for i in df.columns:
    if df[i].dtype=="object":
        print(i)

MARITALSTATUS
EDUCATION
GENDER
last_prod_enq2
first_prod_enq2
Approved_Flag


In [28]:
print(df["MARITALSTATUS"].value_counts())
print(df["EDUCATION"].value_counts())
print(df["GENDER"].value_counts())
print(df["last_prod_enq2"].value_counts())
print(df["first_prod_enq2"].value_counts())
print(df["Approved_Flag"].value_counts())

MARITALSTATUS
Married    37724
Single     13572
Name: count, dtype: int64
EDUCATION
GRADUATE          16655
12TH              14458
SSC                9268
UNDER GRADUATE     5491
OTHERS             2915
POST-GRADUATE      2240
PROFESSIONAL        269
Name: count, dtype: int64
GENDER
M    45211
F     6085
Name: count, dtype: int64
last_prod_enq2
others          20808
ConsumerLoan    17782
PL               7955
CC               2338
AL               1511
HL                902
Name: count, dtype: int64
first_prod_enq2
others          28095
ConsumerLoan    11852
PL               4885
AL               2869
CC               2187
HL               1408
Name: count, dtype: int64
Approved_Flag
P2    32162
P3     7449
P4     5882
P1     5803
Name: count, dtype: int64


In [31]:
# chi - square test

for i in ['MARITALSTATUS','EDUCATION','GENDER','last_prod_enq2','first_prod_enq2']:
    chi2, pval,_, _ =chi2_contingency(pd.crosstab(df[i], df["Approved_Flag"]))
    print(i, ":->", pval)

MARITALSTATUS :-> 2.7347247145640217e-257
EDUCATION :-> 6.772122013692765e-38
GENDER :-> 0.00025924146444508505
last_prod_enq2 :-> 0.0
first_prod_enq2 :-> 0.0


In [32]:
# simce all the categorical features hace pval < 0.05
# we will accept all

In [33]:
numerical_columns = []
for i in df.columns:
    if df[i].dtype!="object" and  i not in ['PROSPECTID', 'Approved_Flag']:
        numerical_columns.append(i)

In [37]:
vif_data = df[numerical_columns]
total_columns = vif_data.shape[1]
columns_to_be_kept = []
column_index=0


for i in range(0, total_columns):

    vif_value = variance_inflation_factor(vif_data, column_index)
    print(column_index, "---", vif_value)

    if vif_value <= 6:
        columns_to_be_kept.append(numerical_columns[i])
        column_index = column_index+1

    else:
        vif_data = vif_data.drop([numerical_columns[i]], axis=1)    

d:\data science project\Cradit Risk  Project\.ven\lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 --- inf


d:\data science project\Cradit Risk  Project\.ven\lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 --- inf
0 --- 10.967318271504595
0 --- 8.280431943361068
0 --- 6.428482253706429
0 --- 5.510394576267302
1 --- 2.4900436238094183


d:\data science project\Cradit Risk  Project\.ven\lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


2 --- inf
2 --- 1813.189446652738
2 --- 8.260638988187743
2 --- 3.6703254239467227
3 --- 5.6942164467767675
4 --- 4.970645827346293
5 --- 2.002032552173903


d:\data science project\Cradit Risk  Project\.ven\lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


6 --- inf
6 --- 4.788323772675488
7 --- 21.142530080727486
7 --- 32.035916588478756
7 --- 4.452092318882433
8 --- 2.9986771147212212
9 --- 2.7991742557496417
10 --- 4.084955530280214
11 --- 2.180147515711328
12 --- 5.113889440000773
13 --- 3.247544480100841
14 --- 1.2747264556566849
15 --- 8.005890070879005
15 --- 5.397672188311246


d:\data science project\Cradit Risk  Project\.ven\lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


16 --- inf
16 --- 7.274564152451663
16 --- 1.4153535674199744
17 --- 8.503819690047568
17 --- 1.6294794750057182
18 --- 7.078919419257034
18 --- 15.410978252728741
18 --- 1.83632439761608
19 --- 1.565076628611904
20 --- 2.5460982721382064
21 --- 3.1032891116487993
22 --- 2.194287797978664
23 --- 7.383989877498994
23 --- 2.07514825604116
24 --- 2.726449304987255
25 --- 6.293135612441871
25 --- 2.7102677871447955
26 --- 4.923780392902992
27 --- 347440509.5926095
27 --- 2991543440.4349923
27 --- 11936689705.599522
27 --- 1408975611.9399073
27 --- 1782848308.7841969
27 --- 5336252451.079604
27 --- 494571895.8992288
27 --- 8494.670892803804
27 --- 666304530.2187972
27 --- 783906969.1539816
27 --- 1.3267888481797092
28 --- 21.7338293165276
28 --- 2.8690860362884307
29 --- 3.394057799688926
30 --- 9.35823325021184
30 --- 6.988945461891277
30 --- 1.0053521595659223
31 --- 3.0721346286046014
32 --- 2.727775618650995
33 --- 20.732955972009012
33 --- 16.17170247740565
33 --- 1.432520794564997
34 

In [38]:
# check Anove for calumns_to_be_kept

from scipy.stats import f_oneway

columns_to_be_kept_numerical = []

for i in columns_to_be_kept:
    a=list(df[i])
    b=list(df["Approved_Flag"])


    group_P1 = [value for value, group in zip(a,b) if group =="P1"]
    group_P2 = [value for value, group in zip(a,b) if group =="P2"]
    group_P3 = [value for value, group in zip(a,b) if group =="P3"]
    group_P4 = [value for value, group in zip(a,b) if group =="P4"]

    f_statistic, p_value, = f_oneway(group_P1, 
                                     group_P2, 
                                     group_P3, 
                                     group_P4 )
    
    if p_value < 0.05:
        columns_to_be_kept_numerical.append(i)

In [39]:
columns_to_be_kept_numerical

['pct_tl_open_L6M',
 'pct_tl_closed_L6M',
 'Tot_TL_closed_L12M',
 'pct_tl_open_L12M',
 'pct_tl_closed_L12M',
 'Tot_Missed_Pmnt',
 'CC_TL',
 'Home_TL',
 'PL_TL',
 'Secured_TL',
 'Unsecured_TL',
 'Other_TL',
 'Age_Oldest_TL',
 'Age_Newest_TL',
 'time_since_recent_payment',
 'max_recent_level_of_deliq',
 'num_deliq_6_12mts',
 'num_times_60p_dpd',
 'num_std_12mts',
 'num_sub',
 'num_sub_12mts',
 'num_dbt',
 'num_dbt_12mts',
 'num_lss',
 'recent_level_of_deliq',
 'enq_L3m',
 'NETMONTHLYINCOME',
 'Time_With_Curr_Empr',
 'CC_Flag',
 'PL_Flag',
 'pct_PL_enq_L6m_of_ever',
 'pct_CC_enq_L6m_of_ever',
 'HL_Flag',
 'GL_Flag']

In [40]:
features = columns_to_be_kept_numerical +['MARITALSTATUS',
                                          'EDUCATION',
                                          'GENDER',
                                          'last_prod_enq2',
                                          'first_prod_enq2']
df = df[features+["Approved_Flag"]]

In [41]:
df

,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_open_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,...,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,MARITALSTATUS,EDUCATION,GENDER,last_prod_enq2,first_prod_enq2,Approved_Flag
0,0.000,0.00,0,0.000,0.00,0,0,0,4,1,...,0.0,0.0,1,0,Married,12TH,M,PL,PL,P2
1,0.000,0.00,0,1.000,0.00,0,0,0,0,0,...,0.0,0.0,0,0,Single,GRADUATE,F,ConsumerLoan,ConsumerLoan,P2
2,0.125,0.00,0,0.250,0.00,1,0,0,0,2,...,0.0,0.0,1,0,Married,SSC,M,ConsumerLoan,others,P2
3,1.000,0.00,0,1.000,0.00,1,0,0,0,0,...,0.0,0.0,0,0,Married,SSC,M,others,others,P2
4,0.000,0.00,0,0.000,0.00,0,0,0,0,3,...,0.0,0.0,0,0,Married,POST-GRADUATE,M,AL,AL,P1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51291,0.333,0.00,0,0.333,0.00,0,0,0,0,0,...,0.0,0.0,0,0,Married,12TH,M,ConsumerLoan,ConsumerLoan,P4
51292,0.000,0.25,1,0.500,0.25,0,0,0,0,2,...,0.0,0.0,0,0,Married,SSC,M,others,others,P1
51293,0.500,0.50,1,1.000,0.50,0,0,0,0,0,...,1.0,0.0,0,0,Married,SSC,M,ConsumerLoan,others,P3
51294,0.000,0.00,1,0.500,0.50,0,0,0,0,0,...,0.0,0.0,0,0,Single,UNDER GRADUATE,F,ConsumerLoan,others,P2


In [42]:
print(df['MARITALSTATUS'].unique())
print(df['EDUCATION'].unique())
print(df['GENDER'].unique())
print(df['last_prod_enq2'].unique())
print(df['first_prod_enq2'].unique())

['Married' 'Single']
['12TH' 'GRADUATE' 'SSC' 'POST-GRADUATE' 'UNDER GRADUATE' 'OTHERS'
 'PROFESSIONAL']
['M' 'F']
['PL' 'ConsumerLoan' 'others' 'AL' 'CC' 'HL']
['PL' 'ConsumerLoan' 'others' 'AL' 'HL' 'CC']


In [43]:
# EDUCATION 
# SSC : 1
# 12th : 2
#GRADUATE : 3
# UNDER GRADUATE : 3
# POST GRADUATE : 4
# OTHER :1
# PROFESSUINAL :3

In [44]:
df.loc[df["EDUCATION"] == "SSC", ['EDUCATION']] =1
df.loc[df["EDUCATION"] == "12TH", ['EDUCATION']] =2
df.loc[df["EDUCATION"] == "GRADUATE", ['EDUCATION']] =3
df.loc[df["EDUCATION"] == "UNDER GRADUATE", ['EDUCATION']] =3
df.loc[df["EDUCATION"] == "POST-GRADUATE", ['EDUCATION']] =4
df.loc[df["EDUCATION"] == 'OTHERS', ['EDUCATION']] =1
df.loc[df["EDUCATION"] == "PROFESSIONAL", ['EDUCATION']] =3

In [45]:
print(df["EDUCATION"].value_counts())
df["EDUCATION"] = df["EDUCATION"].astype(int)
df.info()

EDUCATION
3    22415
2    14458
1    12183
4     2240
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51296 entries, 0 to 51295
Data columns (total 40 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   pct_tl_open_L6M            51296 non-null  float64
 1   pct_tl_closed_L6M          51296 non-null  float64
 2   Tot_TL_closed_L12M         51296 non-null  int64  
 3   pct_tl_open_L12M           51296 non-null  float64
 4   pct_tl_closed_L12M         51296 non-null  float64
 5   Tot_Missed_Pmnt            51296 non-null  int64  
 6   CC_TL                      51296 non-null  int64  
 7   Home_TL                    51296 non-null  int64  
 8   PL_TL                      51296 non-null  int64  
 9   Secured_TL                 51296 non-null  int64  
 10  Unsecured_TL               51296 non-null  int64  
 11  Other_TL                   51296 non-null  int64  
 12  Age_Oldest_TL         

C:\Users\Sande\AppData\Local\Temp\ipykernel_19872\2544825147.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["EDUCATION"] = df["EDUCATION"].astype(int)


In [46]:
df_encoded = pd.get_dummies(df, columns=['MARITALSTATUS', 'GENDER', 'last_prod_enq2', 'first_prod_enq2'])

In [47]:
df_encoded.info()
k = df_encoded.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51296 entries, 0 to 51295
Data columns (total 52 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   pct_tl_open_L6M               51296 non-null  float64
 1   pct_tl_closed_L6M             51296 non-null  float64
 2   Tot_TL_closed_L12M            51296 non-null  int64  
 3   pct_tl_open_L12M              51296 non-null  float64
 4   pct_tl_closed_L12M            51296 non-null  float64
 5   Tot_Missed_Pmnt               51296 non-null  int64  
 6   CC_TL                         51296 non-null  int64  
 7   Home_TL                       51296 non-null  int64  
 8   PL_TL                         51296 non-null  int64  
 9   Secured_TL                    51296 non-null  int64  
 10  Unsecured_TL                  51296 non-null  int64  
 11  Other_TL                      51296 non-null  int64  
 12  Age_Oldest_TL                 51296 non-null  int64  
 13  A

In [48]:
pd.set_option("display.max_columns", None)
df_encoded

,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_open_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL,time_since_recent_payment,max_recent_level_of_deliq,num_deliq_6_12mts,num_times_60p_dpd,num_std_12mts,num_sub,num_sub_12mts,num_dbt,num_dbt_12mts,num_lss,recent_level_of_deliq,enq_L3m,NETMONTHLYINCOME,Time_With_Curr_Empr,CC_Flag,PL_Flag,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,EDUCATION,Approved_Flag,MARITALSTATUS_Married,MARITALSTATUS_Single,GENDER_F,GENDER_M,last_prod_enq2_AL,last_prod_enq2_CC,last_prod_enq2_ConsumerLoan,last_prod_enq2_HL,last_prod_enq2_PL,last_prod_enq2_others,first_prod_enq2_AL,first_prod_enq2_CC,first_prod_enq2_ConsumerLoan,first_prod_enq2_HL,first_prod_enq2_PL,first_prod_enq2_others
0,0.000,0.00,0,0.000,0.00,0,0,0,4,1,4,0,72,18,549,29,0,0,11,0,0,0,0,0,29,0,51000,114,0,1,0.0,0.0,1,0,2,P2,True,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False
1,0.000,0.00,0,1.000,0.00,0,0,0,0,0,1,0,7,7,47,0,0,0,0,0,0,0,0,0,0,0,19000,50,0,0,0.0,0.0,0,0,3,P2,False,True,True,False,False,False,True,False,False,False,False,False,True,False,False,False
2,0.125,0.00,0,0.250,0.00,1,0,0,0,2,6,0,47,2,302,25,8,0,10,0,0,0,0,0,25,0,18,191,0,0,0.0,0.0,1,0,1,P2,True,False,False,True,False,False,True,False,False,False,False,False,False,False,False,True
3,1.000,0.00,0,1.000,0.00,1,0,0,0,0,1,1,5,5,-99999,0,0,0,5,0,0,0,0,0,0,-99999,10000,246,0,0,0.0,0.0,0,0,1,P2,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True
4,0.000,0.00,0,0.000,0.00,0,0,0,0,3,0,2,131,32,583,0,0,0,16,0,0,0,0,0,0,0,15000,75,0,0,0.0,0.0,0,0,4,P1,True,False,False,True,True,False,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51291,0.333,0.00,0,0.333,0.00,0,0,0,0,0,3,1,24,5,15,24,0,0,0,0,0,0,0,0,24,1,18500,249,0,0,0.0,0.0,0,0,2,P4,True,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False
51292,0.000,0.25,1,0.500,0.25,0,0,0,0,2,2,0,74,7,57,0,0,0,6,0,0,0,0,0,0,0,25000,186,0,0,0.0,0.0,0,0,1,P1,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True
51293,0.500,0.50,1,1.000,0.50,0,0,0,0,0,2,0,9,5,32,0,0,0,0,0,0,0,0,0,0,2,18000,66,0,0,1.0,0.0,0,0,1,P3,True,False,False,True,False,False,True,False,False,False,False,False,False,False,False,True
51294,0.000,0.00,1,0.500,0.50,0,0,0,0,0,2,0,15,8,58,0,0,0,0,0,0,0,0,0,0,0,12802,54,0,0,0.0,0.0,0,0,3,P2,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False,True


In [49]:
x= df_encoded.drop(["Approved_Flag"], axis = 1)
y= df_encoded["Approved_Flag"]

In [50]:
print(x)
print(y)

       pct_tl_open_L6M  pct_tl_closed_L6M  Tot_TL_closed_L12M  \
0                0.000               0.00                   0   
1                0.000               0.00                   0   
2                0.125               0.00                   0   
3                1.000               0.00                   0   
4                0.000               0.00                   0   
...                ...                ...                 ...   
51291            0.333               0.00                   0   
51292            0.000               0.25                   1   
51293            0.500               0.50                   1   
51294            0.000               0.00                   1   
51295            0.000               0.00                   0   

       pct_tl_open_L12M  pct_tl_closed_L12M  Tot_Missed_Pmnt  CC_TL  Home_TL  \
0                 0.000                0.00                0      0        0   
1                 1.000                0.00                

In [51]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 42)

In [52]:
print(x_train)
print(x_test)
print(y_train)
print(y_test)

       pct_tl_open_L6M  pct_tl_closed_L6M  Tot_TL_closed_L12M  \
40531              0.0                0.0                   1   
10279              0.0                0.0                   0   
48509              0.0                0.0                   0   
27091              0.0                0.0                   0   
23317              0.0                0.0                   0   
...                ...                ...                 ...   
11284              1.0                0.0                   0   
44732              0.0                0.0                   0   
38158              0.0                0.0                   1   
860                0.2                0.0                   0   
15795              0.0                0.0                   0   

       pct_tl_open_L12M  pct_tl_closed_L12M  Tot_Missed_Pmnt  CC_TL  Home_TL  \
40531               0.0                 1.0                0      0        0   
10279               0.5                 0.0                

In [53]:
rf_classifier = RandomForestClassifier(n_estimators= 200, 
                                       random_state=42)
rf_classifier.fit(x_train, y_train)

RandomForestClassifier(n_estimators=200, random_state=42)

In [54]:
y_pred = rf_classifier.predict(x_test)

In [55]:
accuracy = classification_report(y_test, y_pred)
print()
print(f"Accuracy{accuracy}")
print()

precision, recall, f1_score, _ =precision_recall_fscore_support(y_test, y_pred)


for  i , v in enumerate(["p1", "p2", "p3", "p4"]):
    print(f"class{v}:- ")
    print(f"Precision:- {precision[i]}")
    print(f"Recall:- {recall[i]}")
    print(f"F1 Score:- {f1_score[i]}")
    print()


Accuracy              precision    recall  f1-score   support

          P1       0.82      0.72      0.77      1145
          P2       0.82      0.94      0.87      6520
          P3       0.44      0.20      0.27      1471
          P4       0.72      0.71      0.71      1124

    accuracy                           0.78     10260
   macro avg       0.70      0.64      0.66     10260
weighted avg       0.75      0.78      0.76     10260


classp1:- 
Precision:- 0.822
Recall:- 0.7179039301310044
F1 Score:- 0.7664335664335664

classp2:- 
Precision:- 0.8163838032874515
Recall:- 0.936963190184049
F1 Score:- 0.8725273155752339

classp3:- 
Precision:- 0.43796711509715996
Recall:- 0.1991842284160435
F1 Score:- 0.2738317757009346

classp4:- 
Precision:- 0.7193140794223827
Recall:- 0.7090747330960854
F1 Score:- 0.71415770609319

